<a href="https://colab.research.google.com/github/Tensai-Pro/SRI-PracticeTasks/blob/master/BERTopic%20for%20chats/BERTopic%20for%20chats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# !pip install bertopic

In [11]:
# !pip install transformers

In [12]:
# !pip install datasets

In [13]:
# !pip install --upgrade accelerate

In [1]:
import pandas as pd
import numpy as np

# Подготовка данных

In [2]:
data = pd.read_csv('/content/data_web.csv', names=['date', 'from', 'text'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689 entries, 0 to 688
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    689 non-null    object
 1   from    689 non-null    object
 2   text    689 non-null    object
dtypes: object(3)
memory usage: 16.3+ KB


In [3]:
docs = data['text'].tolist()
print(type(docs))
print(docs[:3])

<class 'list'>
['Настройка https на localhost . ', 'Новый сервис капчи от яндекса на замену сервису ReCAPTCHA от Гугла. Кстати прикольно, не думал, что капча это вариация на тему теста Тьюринга. Если не знаете что это за тест, рекомендую погуглить, развивает кругозор,   . ', 'Введение в svg анимацию доя верстальщиков  . ']


# Подготовка модели BERTopic

In [4]:
from bertopic import BERTopic

In [5]:
from transformers.pipelines import pipeline

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
import nltk;
nltk.download('stopwords');
from nltk.corpus import stopwords;
stopwords = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)

Чистая модель rubert_tiny

In [9]:
embedding_model = pipeline('feature-extraction', model='cointegrated/rubert-tiny2')

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
topic_model = BERTopic(language="multilingual", embedding_model=embedding_model,
                       vectorizer_model=vectorizer_model,
                       calculate_probabilities=True, verbose=True)

topics, probs = topic_model.fit_transform(docs)

100%|██████████| 689/689 [00:06<00:00, 106.98it/s]
2023-05-13 15:54:05,722 - BERTopic - Transformed documents to Embeddings
2023-05-13 15:54:18,606 - BERTopic - Reduced dimensionality
2023-05-13 15:54:18,655 - BERTopic - Clustered reduced embeddings


# Извлечение тем

In [11]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name
0,0,664,0_это_коллеги_очень_спасибо
1,1,25,1_позиция абсолютная_позиция_группы позиция_са...


In [12]:
topic_model.get_topic(0)

[('это', 0.029800860487000688),
 ('коллеги', 0.014868404811310723),
 ('очень', 0.013435492129944578),
 ('спасибо', 0.012331723320136684),
 ('js', 0.009640960766043613),
 ('сколько', 0.009241183131225064),
 ('всем', 0.008836974315738884),
 ('который', 0.008428086315968074),
 ('google', 0.008014241902650541),
 ('проект', 0.008014241902650541)]

In [13]:
topic_model.update_topics(docs, n_gram_range=(1, 2))

In [14]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name
0,0,664,0_не_на_что_как
1,1,25,1_позиция абсолютная_позиция_группы позиция_са...


# Rubert_tiny

## Подготовка данных из huggingface hub

In [15]:
from datasets import load_dataset

In [22]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.cache/huggingface

In [23]:
dataset = load_dataset('YuRayne/sri_pracrice_2')

  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
# dataset['train']

In [25]:
len(dataset['train']['date'])

689

In [26]:
dataset['train'][10]

{'date': '2022-07-04 09:30:50',
 'from': 'Антон Евтюшкин',
 'text': 'Статья обзор сервиса Github Copilot, искусственного интеллекта, который помогает вам писать код.   . '}

## Переобучение модели

In [27]:
from transformers import (AutoModel, AutoModelForMaskedLM,
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments)

In [28]:
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')
model = AutoModelForMaskedLM.from_pretrained('cointegrated/rubert-tiny2')

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
# разбивает датасет на батчи
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# берет каждую строчку текстового файла
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='/content/examples_web.txt',
    block_size=128,
)

In [33]:
trained_path = 'train/rubert-tiny2_trained'

training_args = TrainingArguments(
    output_dir=trained_path,
    overwrite_output_dir=True,
    num_train_epochs=2,
    save_steps=5000,
    save_total_limit=2,
    prediction_loss_only=True,
)

In [34]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [35]:
saved_path = 'trained_path/'
trainer.train()
trainer.save_model(saved_path)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


## Как работает AutoTokenizer и AutoModel

In [36]:
import torch

In [37]:
def tokenize(text, tokenizer):
  print()
  # делит на слова (токены)
  print(tokenizer.tokenize(text))

  t = tokenizer(text[0], text[1], padding=True, truncation=True)
  # делит на числа
  print('t:', t)

  encoded_seq = t['input_ids']
  print('encoded_seq:', encoded_seq)

  decoded_seq = tokenizer.decode(encoded_seq)
  print('decoded_seq:', decoded_seq)

tokenize(['привет мир.', 'Как дела?'], tokenizer)


['привет', 'мир', '.', 'Как', 'дела', '?']
t: {'input_ids': [2, 32840, 9311, 18, 3, 7063, 7843, 35, 3], 'token_type_ids': [0, 0, 0, 0, 0, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
encoded_seq: [2, 32840, 9311, 18, 3, 7063, 7843, 35, 3]
decoded_seq: [CLS] привет мир. [SEP] Как дела? [SEP]


In [38]:
def tokenize_batch(text, tokenizer):
  print()

  encoded_inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
  print('encoded_inputs:')
  print(encoded_inputs)

tokenize_batch(['привет.', 'Как у тебя дела?', 'утка ходит'], tokenizer)


encoded_inputs:
{'input_ids': tensor([[    2, 32840,    18,     3,     0,     0,     0],
        [    2,  7063,   331, 29852,  7843,    35,     3],
        [    2, 78415,   872, 38665,     3,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0]])}


Функция из документации к rubert_tiny

In [39]:
model_original = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [40]:
def embed_bert_cls(text, model, tokenizer):
  t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

  with torch.no_grad():
    # то же самое, что и model(**t)
    model_output = model(**{k: v.to(model.device) for k, v in t.items()})

  print(model_output.last_hidden_state.shape)

  embeddings = model_output.last_hidden_state[:, 0, :]
  pooler = model_output.pooler_output
  # print(embeddings.shape)
  # print(pooler.shape)

  # print(embeddings)
  embeddings = torch.nn.functional.normalize(embeddings)
  return embeddings[0].cpu().numpy()

print(embed_bert_cls('привет мир', model_original, tokenizer)[:5])

torch.Size([1, 4, 312])
[ 0.01934799 -0.02039804 -0.00845532 -0.06858292 -0.00167201]


# BERTopic с дообученной моделью векторного представления

## Извлечение векторных представлений

In [76]:
model_custom = AutoModel.from_pretrained('/content/trained_path', output_hidden_states=False)

Some weights of the model checkpoint at /content/trained_path were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /content/trained_path and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']

In [77]:
docs[3]

'Хороший интерактивный мануал по использованию Git  . '

In [78]:
def embed_bert_cls_custom(text, model, tokenizer):
  t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

  with torch.no_grad():
      model_output = model(**{k: v.to(model.device) for k, v in t.items()})

  embeddings = model_output.last_hidden_state[:, 0, :]
  embeddings = torch.nn.functional.normalize(embeddings)
  return embeddings[0].cpu().numpy()

In [79]:
print(embed_bert_cls_custom(docs[0], model_custom, tokenizer).shape)

(312,)


In [80]:
docs_embeddings = [embed_bert_cls_custom(doc, model_custom, tokenizer) for doc in docs]

In [81]:
print(type(docs_embeddings))
print(len(docs_embeddings))
print(docs_embeddings[0].shape)

<class 'list'>
689
(312,)


In [82]:
docs_embeddings = np.array(docs_embeddings)
print(type(docs_embeddings))
print(docs_embeddings.shape)

<class 'numpy.ndarray'>
(689, 312)


## Извлечение тем

In [83]:
bertopic_model = BERTopic(language="multilingual", vectorizer_model=vectorizer_model)
topics, probs = bertopic_model.fit_transform(docs, docs_embeddings)

2023-05-13 17:34:33,219 - BERTopic - Reduced dimensionality
2023-05-13 17:34:33,252 - BERTopic - Clustered reduced embeddings


In [84]:
freq2 = bertopic_model.get_topic_info(); freq2.head(10)

,Topic,Count,Name
0,0,660,0_это_коллеги_очень_спасибо
1,1,29,1_равно_равно голова_верх_аршинов равно


In [86]:
bertopic_model.get_topic(1)

[('равно', 0.7682395726004714),
 ('равно голова', 0.4570559286132247),
 ('верх', 0.4570559286132247),
 ('аршинов равно', 0.4570559286132247),
 ('аршинов', 0.4570559286132247),
 ('группы такая', 0.4570559286132247),
 ('возможность верх', 0.4570559286132247),
 ('верх аршинов', 0.4570559286132247),
 ('равно равно', 0.4570559286132247),
 ('сайтописец', 0.4570559286132247)]

# Twhin model

In [50]:
tokenizer_twhin = AutoTokenizer.from_pretrained('Twitter/twhin-bert-base')
model_twhin = AutoModel.from_pretrained('Twitter/twhin-bert-base')

Some weights of the model checkpoint at Twitter/twhin-bert-base were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at Twitter/twhin-bert-base and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bi

In [51]:
# text_example = "I'm using TwHIN-BERT! #TwHIN-BERT #NLP"
text_example = docs[3]

In [52]:
inputs = tokenizer_twhin.tokenize(text_example)

Здесь нет [CLS] токенов

In [53]:
inputs

['▁Хорош',
 'ий',
 '▁интерактив',
 'ный',
 '▁',
 'ману',
 'ал',
 '▁по',
 '▁использован',
 'ию',
 '▁G',
 'it',
 '▁',
 '.']

In [54]:
inputs = tokenizer_twhin(text_example, return_tensors='pt')

In [55]:
inputs

{'input_ids': tensor([[     0, 189611,    983, 175419,   2192,      6, 131816,   3844,    129,
         136383,   4256,    527,    217,      6,      5,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [56]:
# эта строчка — то, что действительно передаётся в модель через **
# outputs = model(inputs['input_ids'], inputs['attention_mask'])
outputs = model_twhin(**inputs)

In [57]:
print(type(outputs))

<class 'transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions'>


In [58]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [59]:
outputs.last_hidden_state.shape

torch.Size([1, 16, 768])

In [60]:
weighted_outputs = outputs.last_hidden_state.mean(axis=1)

In [61]:
weighted_outputs.shape

torch.Size([1, 768])

## Извлечение векторных представлений

In [62]:
def embed_weighted(text, model, tokenizer):
  tokens = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

  with torch.no_grad():
      outputs = model(**tokens)

  embeddings = outputs.last_hidden_state.mean(axis=1)
  # print(embeddings[0][:5])
  embeddings = torch.nn.functional.normalize(embeddings)
  # print(embeddings[0][:5])
  return embeddings[0].numpy()

In [63]:
embed = embed_weighted(text_example, model_twhin, tokenizer_twhin)
print(embed[:5])
print(embed.shape)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[-0.01275232  0.06367259  0.01556224  0.11286029  0.01879516]
(768,)


In [64]:
docs[3]

'Хороший интерактивный мануал по использованию Git  . '

In [65]:
docs_embeddings = [embed_weighted(doc, model_twhin, tokenizer_twhin) for doc in docs]

In [66]:
print(len(docs_embeddings))
print(docs_embeddings[0].shape)

689
(768,)


In [67]:
docs_embeddings = np.array(docs_embeddings)
print(docs_embeddings.shape)

(689, 768)


## Извлечение тем

In [68]:
bertopic_model_twhin = BERTopic(language="multilingual", vectorizer_model=vectorizer_model)
topics, probs = bertopic_model_twhin.fit_transform(docs, docs_embeddings)

2023-05-13 16:20:55,082 - BERTopic - Reduced dimensionality
2023-05-13 16:20:55,113 - BERTopic - Clustered reduced embeddings


In [69]:
freq3 = bertopic_model_twhin.get_topic_info(); freq3.head(10)

,Topic,Count,Name
0,0,617,0_это_коллеги_спасибо_очень
1,1,44,1_via_generated via_via news_generated
2,2,28,2_это_лампочку_нужно_сколько


In [73]:
bertopic_model_twhin.get_topic(2)

[('это', 0.023675638821807406),
 ('лампочку', 0.020893238098689725),
 ('нужно', 0.0204006820351221),
 ('сколько', 0.019767176934690895),
 ('поменять лампочку', 0.01839612063950907),
 ('поменять', 0.01839612063950907),
 ('нужно поменять', 0.017115514521371337),
 ('код', 0.013431367349157678),
 ('российской', 0.013119836427612039),
 ('российской федерации', 0.013119836427612039)]